In [1]:
# importing dependenciesimport matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
import csv
import gmaps
import gmaps.datasets
from scipy.stats import linregress
from datetime import datetime
from api_keys import g_key

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

# importing 2018 data from csv downloaded from NYC DMV website
csv_file = "dmv_2018_data"


In [2]:
# Create DMV dataframe
dmv_df = pd.read_csv('dmv_2018_data.csv')
dmv_df.dropna(inplace = True)
dmv_df.head()

,CRASH DATE,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,YEAR,MONTH,MONTH NAME
0,07/04/2018,BROOKLYN,11225,40.663456,-73.958360,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,Sedan,Station Wagon/Sport Utility Vehicle,2018,7,Jul
1,07/18/2018,BRONX,10473,40.816906,-73.847550,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,Taxi,Station Wagon/Sport Utility Vehicle,2018,7,Jul
3,07/15/2018,QUEENS,11358,40.760784,-73.806770,0.0,0.0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Unspecified,Sedan,Station Wagon/Sport Utility Vehicle,2018,7,Jul
4,07/22/2018,BRONX,10453,40.861862,-73.912820,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,Station Wagon/Sport Utility Vehicle,Sedan,2018,7,Jul
6,07/20/2018,QUEENS,11385,40.706665,-73.916824,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,2018,7,Jul


In [3]:
grouby_data = dmv_df.groupby('ZIP CODE')
grouby_dmv_zipcode = grouby_data.size()
grouby_dmv_lat_lon = grouby_data.first()
grouby_dmv_sum = grouby_data.sum()
groupby_dmv_zipcode_df = pd.DataFrame({
    "Total Accident": grouby_dmv_zipcode,
    "Lat": grouby_dmv_lat_lon['LATITUDE'],
    "Lon": grouby_dmv_lat_lon['LONGITUDE'],
    "Num of People Injured": grouby_dmv_sum['NUMBER OF PERSONS INJURED'],
    "Num of People Killed": grouby_dmv_sum['NUMBER OF PERSONS KILLED']
})
groupby_dmv_zipcode_df.sort_values(by=["Total Accident"], ascending=False, inplace=True)
top_ten_zipcode = groupby_dmv_zipcode_df.head(20)
top_ten_zipcode

,Total Accident,Lat,Lon,Num of People Injured,Num of People Killed
ZIP CODE,,,,,
11207,1581,40.687460,-73.906494,509.0,2.0
11101,1355,40.742550,-73.931160,280.0,0.0
11234,1201,40.632706,-73.934180,407.0,1.0
10001,1197,40.751026,-73.996620,95.0,0.0
10016,1189,40.744778,-73.975890,167.0,0.0
11385,1175,40.706665,-73.916824,277.0,1.0
10013,1159,40.717007,-73.996710,131.0,1.0
11201,1121,40.695908,-73.983220,178.0,0.0
10022,1101,40.759308,-73.965300,114.0,0.0


In [4]:
locations = top_ten_zipcode[['Lat', 'Lon']]

In [5]:
# Create a marker_layer using the poverty list to fill the info box top zipcodes
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=[f"Total Accident: {row['Total Accident']} People Injured: {row['Num of People Injured']}, Killed: {row['Num of People Killed']}" for index, row in top_ten_zipcode.iterrows()])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
heatmap_locations = top_ten_zipcode[['Lat', 'Lon']]
heatweights = top_ten_zipcode['Total Accident']
# Plost as a Hybrid map
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(heatmap_locations, weights=heatweights, 
                                 dissipating=False, max_intensity=1581,
                                 point_radius=0.1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
top_killed_df = groupby_dmv_zipcode_df.sort_values(by=["Num of People Injured"], ascending=False).head(20)
top_killed_df

,Total Accident,Lat,Lon,Num of People Injured,Num of People Killed
ZIP CODE,,,,,
11207,1581,40.687460,-73.906494,509.0,2.0
11236,983,40.635740,-73.908530,440.0,2.0
11234,1201,40.632706,-73.934180,407.0,1.0
11434,949,40.692066,-73.765400,378.0,1.0
11203,827,40.638447,-73.928986,354.0,0.0
10469,871,40.877132,-73.838620,348.0,1.0
11208,928,40.670063,-73.879800,326.0,1.0
11212,777,40.667366,-73.922740,325.0,1.0
10466,893,40.894660,-73.861370,315.0,1.0


In [8]:

heatmap_locations = top_killed_df[['Lat', 'Lon']]
heatweights = top_killed_df['Total Accident']
# Plost as a Hybrid map
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(heatmap_locations, weights=heatweights, 
                                 dissipating=False, max_intensity=509.0,
                                 point_radius=0.1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

![08-Terrain_Map.png](../Images/08-Terrain_Map.png)